In [1]:
import cv2
import numpy as np

In [2]:
def dense_optical_flow(method, video_path, params=[], to_gray=True):
    
    # Read the video
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, old_frame = cap.read()

    # Create HSV & make value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Preprocessing for exact method
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    
    j=0
    while True:
        
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break
            
        # Preprocessing for exact method
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
            
        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)
        #print(flow)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Use Hue and Saturation to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert HSV image into BGR for demo
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.imshow("frame", frame_copy)
        cv2.imshow("optical flow", bgr)
       
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        
        name = f'./opticalhsv_{j}.png'
        cv2.imwrite(name,bgr)

        pixel_y = 1400
        pixel_x = 1050
        magnitude = np.linalg.norm(flow[pixel_x,pixel_y])
        print(magnitude)
        
        old_frame = new_frame 
        j+=1

In [3]:
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # default Farneback's algorithm parameters
videopath = 'C:/Users/Margo/GOES16/GOES16_11212019_20_21Z_10.mp4'
frames = dense_optical_flow(method, videopath, params, to_gray=True)

36.187805
51.580097
39.969852
3.2626405e-05
1.6027392e-10
2.128687e-06
39.742992
2.1607653e-05
1.834173e-09
1.7165985e-05
25.194008
40.117023


In [4]:
cv2.destroyAllWindows()